In [26]:
import pandas as pd
import pycountry
import c3aidatalake

In [27]:
df = pd.read_csv('./data/_country_vaccinations.csv')

In [28]:
countries = list(set(df['country']))
dates = list(set(df['date']))
dates.sort()

In [31]:
new_headers = ['date', 'daily_vaccinations', 'people_fully_vaccinated']
global_headers = ['country', 'date', 'daily_vaccinations', 'people_fully_vaccinated', 'vaccines']

In [60]:
for i, country in enumerate(countries):
    country_df = df[df['country'] == country][new_headers]
    country_dates = set(country_df['date'])
    today = pd.Timestamp.now().strftime("%Y-%m-%d")
    casecounts = c3aidatalake.evalmetrics(
        "outbreaklocation",
        {
            "spec" : {
                "ids" : ["Canada"],
                "expressions" : ["JHU_ConfirmedCases"],
                "start" : "2020-12-13",
                "end" : today,
                "interval" : "DAY",
            }
        }
    )
    print(casecounts.iloc[[26]]['dates'].to_string(index=False), country_dates, str(casecounts.iloc[[15]]['dates']) in country_dates)
    casecounts = [casecounts.iloc[[i]] if not i else casecounts.iloc[[i]] - casecounts.iloc[[i-1]] for i in range(len(casecounts)) if casecounts.iloc[[i]]['dates'] in country_dates]
    country_df['cases'] = casecounts
    country_df.to_csv(f'./data/{country}_vaccinations.csv', index=False, header=True)

2020-12-28 {'2021-01-08', '2021-01-07', '2021-02-02', '2021-01-25', '2021-01-01', '2021-01-06', '2020-12-30', '2021-01-17', '2021-01-30', '2021-01-19', '2021-01-09', '2021-01-26', '2021-02-11', '2021-01-14', '2021-01-20', '2021-01-16', '2021-01-23', '2021-02-13', '2021-02-04', '2021-01-04', '2021-01-10', '2021-02-12', '2021-02-06', '2021-02-15', '2020-12-31', '2021-02-07', '2021-01-15', '2021-01-11', '2021-01-12', '2021-01-27', '2021-01-31', '2021-01-03', '2021-01-05', '2021-01-28', '2021-01-21', '2021-02-03', '2021-01-02', '2021-02-01', '2021-01-29', '2021-01-22', '2021-02-14', '2021-02-08', '2021-02-05', '2021-02-10', '2021-01-24', '2021-02-09', '2021-01-13', '2021-01-18'} False


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [41]:
data = []
for i, date in enumerate(dates):
    cur_dates_df = df[df['date']==date][new_headers]
    sum_column = list(cur_dates_df.sum(axis=0, numeric_only=True))
    sum_column.insert(0, date)
    data.append(sum_column)
dates_df = pd.DataFrame(data, columns=['date', 'daily_vaccinations', 'people_fully_vaccinated'])
dates_df.head

# df.to_csv('./data/Global_vaccinations.csv', index=False, header=True)

<bound method NDFrame.head of           date  daily_vaccinations  people_fully_vaccinated
0   2020-12-13                 0.0                      0.0
1   2020-12-14            167881.0                      0.0
2   2020-12-15            167881.0                      0.0
3   2020-12-16            358738.0                      0.0
4   2020-12-17            358738.0                      0.0
..         ...                 ...                      ...
60  2021-02-11           4450435.0               22097097.0
61  2021-02-12           4714945.0               22297333.0
62  2021-02-13           4793876.0               24101382.0
63  2021-02-14           4769998.0               25916204.0
64  2021-02-15           1868502.0                6790959.0

[65 rows x 3 columns]>

In [21]:
# Fetch facts about Germany
locations = c3aidatalake.fetch(
    "outbreaklocation",
    {
        "spec" : {
            "filter" : "id == 'Germany'"
        }
    }
)

locations

,latestTotalPopulation,populationOfAllChildren,id,name,typeIdent,hospitalPrediction.timestamp,location.value.id,location.timestamp
0,71541906.0,71541906.0,Germany,Germany,EP_LOC,2021-02-20T08:01:12Z,Germany,2021-02-21T06:00:13Z
